In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
weather = pd.read_csv("../data/citylearn_challenge_2021/weather.csv")
#carbon = pd.read_csv("../data/citylearn_challenge_2021/carbon_intensity.csv")

b5 = pd.read_csv("../data/citylearn_challenge_2021/Building_5.csv").assign(
        datetime = pd.date_range(start='2008-01-02 01:00:00', end='2012-01-01', freq='h')
)
b5 = (b5.drop(columns=["Heating Load [kWh]"])
.assign(holiday = b5["Day Type"] == 8,)
        #datetime = pd.date_range(start='2008-01-02 01:00:00', end='2012-01-01', freq='h'))
.assign(workday = (b5.datetime.dt.weekday >= 1) & (b5.datetime.dt.weekday <= 5) & (b5["Day Type"] != 8) )
.drop(columns=["Month", "Hour", "Day Type"])
.set_index("datetime")
)

b5.head()
b5.shape

In [ ]:
b5.loc[0:24, ["Solar Generation [W/kW]"]].plot()
weather.loc[0:24, ["Direct Solar Radiation [W/m2]"]].plot()

In [ ]:
fig, ax = plt.subplots()
ax.plot("Solar Generation [W/kW]", data=b5.loc[0:24, :])
ax.plot("Direct Solar Radiation [W/m2]", data=weather.loc[0:24, :])